<a href="https://colab.research.google.com/github/gkeidel/2073-jenkins/blob/master/Projeto_Pipeline_de_ETL_Integrado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# **Extração (E)**

In [ ]:
# carregando o arquivo CSV
df = pd.read_csv("/content/estilo_de_vida.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/content/estilo_de_vida.csv'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Lendo as primeiras 5 linhas do df
df.head()

In [ ]:
# Mostrar todas as colunas (lista nomes)
df.columns

In [ ]:
# salvar em csv
df.to_csv('estilo_de_vida_bronze.csv', index=False)

print('Arquivo salvo com sucesso!')

# **Transformação (T)**

In [ ]:
# faz a leitura do arquivo csv
df_tratado = pd.read_csv('estilo_de_vida_bronze.csv')

df_tratado.head()

In [ ]:
# Escolhendo as colunas a serem trabalhadas
df_tratado = df_tratado[['Gender', 'Age', 'Weight (kg)', 'Fat_Percentage', 'BMI', 'diet_type', 'Workout_Type', 'Workout_Frequency (days/week)', 'Experience_Level']]

df_tratado.head()

In [ ]:
# Renomeando as colunas
df_tratado = df_tratado.rename(columns={
  "Gender" : "genero",
  "Age" : "idade",
  "Weight (kg)" : "peso",
  "Fat_Percentage" : "percentual_de_gordura",
  "BMI" : "imc",
  "diet_type" : "tipo_de_dieta",
  "Workout_Type" : "tipo_de_treino",
  "Workout_Frequency (days/week)" : "frequencia_de_treino_semanal",
  "Experience_Level" : "nivel_de_experiencia"
})

df_tratado.head()

In [ ]:
# Cria lista do genero
df_tratado["genero"].unique().tolist()

In [ ]:
# Cria lista do tipo de dieta
df_tratado["tipo_de_dieta"].unique().tolist()

In [ ]:
# Cria lista do tipo de treino
df_tratado["tipo_de_treino"].unique().tolist()

In [ ]:
# Cria listas do niveol de experiência
df_tratado["nivel_de_experiencia"].unique().tolist()

In [ ]:
# Traduzindo termos Gênero
df_tratado["genero"] = df_tratado["genero"].map({
    "Male": "Masculino",
    "Female": "Feminino"
})

In [ ]:
# Traduzindo termos Tipo de Dieta
df_tratado["tipo_de_dieta"] = df_tratado["tipo_de_dieta"].map({
    "Vegan": "Vegana",
    "Vegetarian": "Vegetariana",
    "Paleo": "Paleolítica",
    "Keto": "Cetogênica",
    "Low-Carb": "Baixo Carboidrato",
    "Balanced": "Balanceada"
})


In [ ]:
# Traduzindo Tipo de Treino
df_tratado["tipo_de_treino"] = df_tratado["tipo_de_treino"].map({
    "Strength": "Força",
    "HIIT": "HIIT",
    "Cardio": "Cardio",
    "Yoga": "Yoga"
})


In [ ]:
# Arredondamento para 1 Casa Decimal
df_tratado["percentual_de_gordura"] = df_tratado["percentual_de_gordura"].round(1)
df_tratado["peso"] = df_tratado["peso"].round(1)

In [ ]:
df_tratado.head()

In [ ]:
# Transformando várias colunas para int
colunas = ['frequencia_de_treino_semanal', 'idade', 'nivel_de_experiencia']
df_tratado[colunas] = df_tratado[colunas].astype(int)

df_tratado.info()

In [ ]:
# Extraindo os números depois do ponto
df_tratado['frequencia_de_treino_semanal'] = df_tratado['frequencia_de_treino_semanal'].astype(str).str.split('.').str[0]
df_tratado['idade'] = df_tratado['idade'].astype(str).str.split('.').str[0]
df_tratado['nivel_de_experiencia'] = df_tratado['nivel_de_experiencia'].astype(str).str.split('.').str[0]

In [ ]:
# Substituição dos Códigos de Experiência por Categorias
df_tratado["nivel_de_experiencia"] = df_tratado["nivel_de_experiencia"].map({
    "1": "Iniciante",
    "2": "Intermediário",
    "3": "Avançado"
})

In [ ]:
df_tratado.head()

In [ ]:
# salvar em csv
df_tratado.to_csv('estilo_de_vida_silver.csv', index=False)

print('Arquivo salvo com sucesso!')

In [ ]:
df_gold = pd.read_csv('estilo_de_vida_silver.csv')

df_gold.head()

In [ ]:
# Função de Classificação do Índice de Massa Corporal
def classificar_imc(imc):
    if imc < 18.5:
        return "Abaixo do Peso"
    elif 18.5 <= imc <= 24.9:
        return "Peso Normal"
    elif 25.0 <= imc <= 29.9:
        return "Sobrepeso"
    elif 30.0 <= imc <= 34.9:
        return "Obesidade Grau I"
    elif 35.0 <= imc <= 39.9:
        return "Obesidade Grau II"
    else:
        return "Obesidade Grau III"

df_gold["classificacao_imc"] = df_gold["imc"].apply(classificar_imc)

df_gold.head()


In [ ]:
# mover para a 6ª posição (índice 5)
colunas = list(df_gold.columns)
colunas.insert(5, colunas.pop(colunas.index("classificacao_imc")))

df_gold = df_gold[colunas]

df_gold.head()

In [ ]:
# salvar em csv
df_gold.to_csv('estilo_de_vida_gold.csv', index=False)

print('Arquivo salvo com sucesso!')

# **Carregamento (L)**

In [ ]:
from sqlalchemy import create_engine

In [ ]:
# cria o banco
engine = create_engine('sqlite:///estilo_de_vida.db')

# salva todas as tabelas no banco criado
df.to_sql('estilo_de_vida_bronze', con=engine, if_exists='replace', index=False)
df_tratado.to_sql('estilo_de_vida_silver', con=engine, if_exists='replace', index=False)
df_gold.to_sql('estilo_de_vida_gold', con=engine, if_exists='replace', index=False)